python setup.py sdist bdist_wheel

twine upload --skip-existing dist/*

In [1]:
import pandas as pd
import ScraperFC as sfc
import traceback

In [ ]:
year = 2020
league = 'La Liga'

In [ ]:
fbref_scraper = sfc.FBRef()
try:
    # Get matches dataframe from FBRef
    filename = fbref_scraper.scrape_matches(year=year, league=league, save=True)
except:
    traceback.print_exc()
fbref_scraper.close()

In [ ]:
elo_scraper = sfc.ClubElo()
# filename = '2017-2018_La Liga_FBRef_matches.csv'
df = pd.read_csv(filename)
clubelo_names = {
    'Alavés':             'Alaves',
    'Aston Villa':        'AstonVilla',
    'Athletic Club':      'Bilbao',
    'Atlético Madrid':    'Atletico',
    'Cardiff City':       'Cardiff',
    'Celta Vigo':         'Celta',
    'Crystal Palace':     'CrystalPalace',
    'Huddersfield Town':  'Huddersfield',
    'La Coruña':          'Depor',
    'Las Palmas':         'LasPalmas',
    'Leeds United':       'Leeds',
    'Leganés':            'Leganes',
    'Leicester City':     'Leicester',
    'Málaga':             'Malaga',
    'Manchester City':    'ManCity',
    'Manchester Utd':     'ManUnited',
    'Newcastle Utd':      'Newcastle',
    'Norwich City':       'Norwich',
    'Rayo Vallecano':     'RayoVallecano',
    'Real Madrid':        'RealMadrid',
    'Real Sociedad':      'Sociedad',
    'Sheffield Utd':      'SheffieldUnited',
    'Stoke City':         'Stoke',
    'Swansea City':       'Swansea',
    'West Brom':          'WestBrom',
    'West Ham':           'WestHam'
}
home_elos = list()
away_elos = list()
broken = False

# Get ELO scores of all teams on match days
for i in df.index:
    print(i, end='\r')
    date = df.loc[i,'Date']
    home_team = df.loc[i,'Home Team']
    away_team = df.loc[i,'Away Team']
#     print(home_team, away_team)

    # Rename teams if needed. ClubElo uses different names than FBRef
    if home_team in clubelo_names.keys():
        home_team = clubelo_names[home_team]
    if away_team in clubelo_names.keys():
        away_team = clubelo_names[away_team]

    home_elo = elo_scraper.scrape_team_on_date(date=date, team=home_team)
    home_elos.append(home_elo)
    away_elo = elo_scraper.scrape_team_on_date(date=date, team=away_team)
    away_elos.append(away_elo)

    if home_elo==-1 or away_elo==-1:
        print('\n', date, home_team, away_team)
        broken = True
        break
        
if not broken:
    # Add ELO scores to dataframe
    df['Home ELO'] = home_elos
    df['Away ELO'] = away_elos

    # Save
    df.to_csv('{}_{}_w_ELO.csv'.format(league.replace(' ',''), year), index=False)